In [68]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/villa/items/{item_id}"

referer = "https://zigbang.com/home/villa/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/villa/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [4]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/villa/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}

In [8]:
geos_seoul = [
    'wydm'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [10]:
from itertools import islice

len(items)

sliceItem = dict(islice(items.items(), 3000))

len(sliceItem)

3000

In [11]:
print(sliceItem)

{'29154502': {}, '29150835': {}, '29249369': {}, '29212722': {}, '28757499': {}, '29051325': {}, '29242680': {}, '29268991': {}, '29223363': {}, '29193635': {}, '29255011': {}, '29191586': {}, '29276313': {}, '29264728': {}, '29233394': {}, '29268795': {}, '29100002': {}, '29241106': {}, '29273621': {}, '29145076': {}, '29251633': {}, '29090385': {}, '29244866': {}, '29266848': {}, '29274800': {}, '29238199': {}, '29248912': {}, '29199925': {}, '29258923': {}, '29119344': {}, '29169286': {}, '29262478': {}, '29271419': {}, '28998398': {}, '29192769': {}, '29076585': {}, '29250671': {}, '29178560': {}, '29236817': {}, '29264966': {}, '29179301': {}, '29150314': {}, '29268031': {}, '29012328': {}, '29275761': {}, '29219474': {}, '29027425': {}, '29163744': {}, '29012146': {}, '28868652': {}, '29268180': {}, '29092714': {}, '29018721': {}, '29254030': {}, '29269590': {}, '29211295': {}, '29265942': {}, '29193838': {}, '29197890': {}, '29275831': {}, '29266143': {}, '29120337': {}, '291441

In [12]:
result = describe_room_list(sliceItem)

In [32]:
len(result)

2899

In [33]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '2층',
  'address1': '서울시 동대문구 용두동',
  'address2': '39-868',
  'address3': None,
  'agent_address1': '서울 성동구 도선동 333-1',
  'agent_comment': '',
  'agent_email': 'gjeotmd@naver.com',
  'agent_lat': 37.56521512991269,
  'agent_lng': 127.03102475402702,
  'agent_local1': '서울시',
  'agent_local2': '성동구',
  'agent_mobile': '010-3186-0819',
  'agent_name': '다온공인중개사사무소(허대승)',
  'agent_no': 0,
  'agent_phone': '02-2296-8607',
  'bjd_code': '',
  'bonbun_code': '39',
  'bubun_code': '868',
  'building': None,
  'building_type': '-',
  'contract': '서울특별시',
  'deposit': 22500,
  'description': '안녕하세요~! \r\n'
                 '신축2년차 풀옵션 투룸 매물을 \r\n'
                 '소개해 드립니다~! \r\n'
                 '\r\n'
                 '✔️ 매물 기본 설명 \r\n'
                 '- 무융자\r\n'
                 '- 90프로 안심대출가능 \r\n'
                 '- 확정일자, 전입신고 가능 \r\n'
                 '- 1금융권 전세자금대출 가능 \r\n'
                 '- 1층 필로티 자주식 주차\r\n'
                 '\r\n'
                 '✔️ 옵션, 크기, 구조, 내부상태 

In [34]:
filtered_result = export_items(result)

In [35]:
len(filtered_result)

1921

In [38]:
def get_string_image(item):
    image = ''
    
    for list in item:
        image += list['url'] + '?w=400&h=300&q=70&a=1,'
        
    return image[:-1]

In [39]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [40]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [41]:
def get_agent_lat(item):
    agent_lat = str(item['agent_lat'])[0:9]
    
    return agent_lat

def get_agent_lng(item):
    agent_lng = str(item['agent_lng'])[0:10]
    
    return agent_lng

In [42]:
geo_based_dict = {}

In [43]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)

    item['lat'] = get_location_lat(item)
    item['lng'] = get_location_lng(item)
    item['image'] = get_string_image(item['images'])
    item['agent_lat'] = get_agent_lat(item)
    item['agent_lng'] = get_agent_lng(item)    

In [44]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '서울시 송파구 잠실동 231-37 ',
  'address1': '서울시 송파구 잠실동',
  'address2': '231-37',
  'address3': None,
  'agent_address1': '서울특별시 송파구 백제고분로39길 27 1층(석촌동)',
  'agent_email': 'cskggw@gmail.com',
  'agent_lat': '37.506610',
  'agent_lng': '127.103671',
  'agent_mobile': '010-7672-9763',
  'agent_name': '바른부동산중개(최원교)',
  'agent_phone': '02-422-5551',
  'building_type': '-',
  'deposit': 25000,
  'description': '- 2014년 준공된 다세대 빌라 투룸 전세 매물입니다.\r\n'
                 '\r\n'
                 '- 옵션 : 에어컨 세탁기 가스렌지 신발장 싱크대.\r\n'
                 '\r\n'
                 '- 위반건축물 등재돼있는 관계로 전세자금 대출은 어렵습니다.\r\n'
                 '\r\n'
                 '- 입주일은 12월중순~말 협의 가능합니다.\r\n'
                 '\r\n'
                 '- 주차1대 가능합니다.\n'
                 '21.11.01에 직방에 등록된 매물입니다.',
  'elevator': '있음',
  'floor': '5층',
  'floor_all': '5층',
  'image': 'https://ic.zigbang.com/ic/items/29150835/1.jpg?w=400&h=300&q=70&a=1,https://ic.zigbang.com/ic/items/29150835/2.jpg?w=400&h=300&q=70&a=1,https:/

In [46]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [69]:
with open('villa.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'image',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'parking',
        'pets_text',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_intro',
        'user_name',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [72]:
import pymysql
import csv
conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8mb4')
curs = conn.cursor()

sql = """insert into villa (address, agent_address, agent_email, agent_lat, agent_lng,
                            agent_mobile, agent_name, agent_phone, building_type, deposit, 
                            description, elevator, floor, floor_all, image,
                            lat, lng, local1, local2, local3,
                            manage_cost, manage_cost_inc, movein_date, near_subways, options,
                            parking, pets, room_direction, room_type, sales_type,
                            service_type, size, size_m2, title, updated_at,
                            user_intro, user_name, url)
                            values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                    %s,%s,%s,%s,%s,%s,%s,%s)"""

f = open('villa.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

next(rd)

for line in rd:
    curs.execute(sql, (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 
                       line[11], line[12], line[13], line[14], line[15], line[16], line[17], line[18], line[19], line[20], 
                       line[21], line[22], line[23], line[24], line[25], line[26], line[27], line[28], line[29], line[30], 
                       line[31], line[32], line[33], line[34], line[35], line[36], line[37]))
conn.commit()
conn.close()
f.close()